In [ ]:
import scipy
from os import listdir
import os
from os.path import isfile, join
import PIL
from PIL import Image
import pydicom
import math 
from pathlib import Path
import copy
import operator
import imageio
import matplotlib.pyplot as plt
import pandas as pd
import csv
import tensorflow as tf
import keras
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import regularizers
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import numpy as np
import glob
from sklearn.model_selection import train_test_split
import pandas as pd
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam# use ADAM if it doesnt work
from tensorflow.keras.optimizers import legacy

In [ ]:
def split_patients(subjects, ratios):

    number_of_patients = len(subjects)
    train_size = int(number_of_patients*ratios[0])
    test_size = number_of_patients - train_size

    val_size = int(train_size*ratios[1])
    train_size -= val_size

    return train_size, val_size, test_size


def process_subjects(list_of_names, table):
    pixel_data = []
    labels = []

    for name in list_of_names:
        matching_files = table.loc[table['name'] == name]

        for _, row in matching_files.iterrows():
            filename = row['filename']
            label = row['label']
            try:
                img = Image.open("/Shared/CipacProcessing/Projects/RF_ML_ENA1/IM1/"+f"{filename}").convert('L')  # Convert image to grayscale
            except FileNotFoundError:
                print(f"File not found: {filename}")
                continue
            
            if img.size == (1024, 1024):
                pixel_array = np.array(img)
                #min-max normalization
                min_value = np.min(pixel_array)
                max_value = np.max(pixel_array)
                pixel_array = (pixel_array - min_value) / (max_value - min_value + 1e-12)
                
                pixel_data.append(pixel_array.reshape(pixel_array.shape[0], pixel_array.shape[1], 1))
                labels.append(label)

    labels_array = np.array([0 if label.lower() == "oc" else 1 for label in labels])
    pixel_array = np.stack(pixel_data, axis=0)  # Stack arrays along a new dimension
    
    return pixel_array, labels_array

In [ ]:
list_files = glob.glob("/Shared/CipacProcessing/Projects/RF_ML_ENA1/IM1/*.jpg")
print(len(list_files))
table_metadata = pd.read_csv("/Shared/CipacProcessing/Projects/RF_ML_ENA1/table.csv")

In [ ]:
subjects = set()

for s in table_metadata["name"]:
    subjects.add(s)

print("Number of patients:", len(subjects))

np.random.seed(2)
subject_list = list(subjects)
np.random.shuffle(subject_list)

ratios = [0.8, 0.2]#train/val
train_size, val_size, test_size = split_patients(subjects, ratios)

subject_train_set = subject_list[:train_size]
subject_val_set = subject_list[train_size:train_size+val_size]
subject_test_set = subject_list[train_size+val_size:]

print("Train size:", train_size)
print("Val size:", val_size)
print("Test size:", test_size)



In [ ]:
pixel_array_train, labels_train = process_subjects(subject_train_set, table_metadata)
pixel_array_val, labels_val = process_subjects(subject_val_set, table_metadata)
pixel_array_test, labels_test = process_subjects(subject_test_set, table_metadata)

zeroList = []
oneList = []
for i in range(len(pixel_array_train)):
    if (labels_train[i] == 0):
        zeroList.append(pixel_array_train[i])
    else:
        oneList.append(pixel_array_train[i])

zeroList = np.array(zeroList)
oneList = np.array(oneList)
oneList = oneList.reshape(oneList.shape[0],oneList.shape[1],oneList.shape[2],1)
zeroList = zeroList.reshape(zeroList.shape[0],zeroList.shape[1],zeroList.shape[2],1)

images0 = zeroList
images1 = oneList

datagen_train = ImageDataGenerator(
    rotation_range=5,  # Small rotations
    width_shift_range=0.1,  # Small horizontal shifts
    height_shift_range=0.1,  # Small vertical shifts
    zoom_range=0.1,  # Slight zoom in and out
    shear_range=0.1,
    #horizontal_flip=True
    # vertical_flip=False, 
    fill_mode='nearest'  # Fill missing pixels with the nearest value
    #preprocessing_function=preprocess_input
)

it0 = datagen_train.flow(images0, batch_size = 1)
it1 = datagen_train.flow(images1, batch_size = 1)

augmented0_train = []
augmented1_train = []

for i in range(len(images0)):
    for j in range(10):
        batch = it0.next()
        image0 = batch[0]
        augmented0_train.append(image0)

for i in range(len(images1)):
    for j in range(10):
        batch = it1.next()
        image1 = batch[0]
        augmented1_train.append(image1)

augmented0_train = np.array(augmented0_train)
augmented1_train = np.array(augmented1_train)

augmented0 = np.concatenate([augmented0_train, images0], axis=0)
augmented1 = np.concatenate([augmented1_train, images1], axis=0)



print(augmented0.shape)
print(augmented1.shape)


In [ ]:
concatenated_train = np.concatenate((augmented1, augmented0), axis=0)
newLabels = np.concatenate((np.ones(572), np.zeros(572)))

indices = np.arange(len(concatenated_train))

np.random.shuffle(indices)

pixel_array_train = concatenated_train[indices]
labels_train = newLabels[indices]

In [ ]:

newXtest = []
for x in pixel_array_test:
    xx = np.empty([x.shape[0], x.shape[1], 3], dtype=np.float32)
    xx[:,:,0] = x[:,:,0]
    xx[:,:,1] = x[:,:,0]
    xx[:,:,2] = x[:,:,0]
    newXtest.append(xx)

newXtest = np.asarray(newXtest)

newXtrain = []
for x in pixel_array_train:
    xx = np.empty([x.shape[0], x.shape[1], 3], dtype=np.float32)
    xx[:,:,0] = x[:,:,0]
    xx[:,:,1] = x[:,:,0]
    xx[:,:,2] = x[:,:,0]
    newXtrain.append(xx)

newXtrain = np.asarray(newXtrain)

newXtarget = []
for x in pixel_array_val:
    xx = np.empty([x.shape[0], x.shape[1], 3], dtype=np.float32)
    xx[:,:,0] = x[:,:,0]
    xx[:,:,1] = x[:,:,0]
    xx[:,:,2] = x[:,:,0]
    newXtarget.append(xx)
    
newXtarget = np.asarray(newXtarget)

print(newXtrain.shape, labels_train.shape)
print(newXtarget.shape, labels_val.shape)
print(newXtest.shape, labels_test.shape)

In [ ]:
print(np.min(newXtrain[0]), np.max(newXtrain[0]))
print(np.min(newXtrain[1]), np.max(newXtrain[1]))
plt.subplot(1,2,1)
plt.title(labels_train[0])
plt.imshow(newXtrain[0], cmap="gray")
plt.axis("off")
plt.subplot(1,2,2)
plt.title(labels_train[1])
plt.imshow(newXtrain[1], cmap="gray")
plt.axis("off")
plt.show()

In [ ]:
from tensorflow.keras import regularizers
model_path_pretrained = "/Shared/CipacProcessing/Projects/RF_ML_ENA1/05-Scripts/RadImageNet-ResNet50_notop.h5"

pretrained_model = tf.keras.applications.ResNet50(include_top=False,
                        input_shape=None,
                        pooling='avg',
                        weights=model_path_pretrained)

pretrained_model.trainable = False
# added new:
# fine_tune_at = len(pretrained_model.layers) // 2
# for layer in pretrained_model.layers[:fine_tune_at]:
#     layer.trainable = False

input_A = keras.layers.Input(shape=[1024, 1024, 3]) # maybe change to 1024, 1024, 1
input_A = preprocess_input(input_A)

fc = keras.layers.Flatten()(pretrained_model(input_A, training=False))
fc = tf.keras.layers.Dropout(0.5)(fc)
fc = keras.layers.Dense(30, activation=tf.keras.layers.LeakyReLU(0.1))(fc)
fc = tf.keras.layers.Dropout(0.4)(fc)
fc = keras.layers.Dense(1, activation="sigmoid")(fc)

# fc = keras.layers.Flatten()(pretrained_model(input_A, training=False))
# fc = tf.keras.layers.Dropout(0.5)(fc)
# fc = keras.layers.Dense(100, activation=tf.keras.layers.LeakyReLU(0.3), kernel_regularizer=regularizers.l2(0.001))(fc)
# fc = tf.keras.layers.Dropout(0.3)(fc)
# fc = keras.layers.Dense(1, activation="sigmoid")(fc)

model = keras.Model(inputs=[input_A], outputs=[fc])


opt = legacy.Adam(learning_rate=1e-4, decay=1e-7)

# Define the callbacks
#earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='auto')
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-8)

model_chck_point_path = "/Shared/CipacProcessing/Projects/RF_ML_ENA1/mymodel_capstone.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(model_chck_point_path, mode = 'auto', monitor='val_loss',verbose=0, save_best_only=True)

# Compile the model with the optimizer
model.compile(loss=[tf.keras.losses.BinaryCrossentropy(label_smoothing=0.1)], optimizer=opt)

In [ ]:
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input

datagen_test = ImageDataGenerator(

)

In [ ]:
data_ = datagen_train.flow(newXtrain[:2], labels_train[:2],batch_size=2)
count = 0
for img,label in data_:
    print(img.shape)
    plt.figure()
    plt.subplot(1,2,1)
    plt.title(label[0])
    plt.imshow(img[0], cmap="gray")
    plt.axis("off")
    plt.subplot(1,2,2)
    plt.title(label[1])
    plt.imshow(img[1], cmap="gray")
    plt.axis("off")
    count += 1

    if count == 10:
        break
plt.show()

In [ ]:
# batch_size_ = 2
# hist = model.fit(datagen_train.flow(pixel_array_train[:2], labels_train[:2],batch_size=batch_size_, shuffle=True), 
#                  steps_per_epoch=math.ceil(len(pixel_array_train[:2]) / batch_size_),
#                  epochs=25, 
#                  verbose = 1, 
#                  validation_data = datagen_test.flow(pixel_array_val[:2], labels_val[:2],batch_size=2),
#                  callbacks = [checkpoint, reduce_lr])


batch_size_ = 20
hist = model.fit(datagen_train.flow(newXtrain, labels_train,batch_size=batch_size_, shuffle=True), 
                 steps_per_epoch=math.ceil(len(newXtrain) / batch_size_),
                 epochs=10, 
                 verbose = 1, 
                 validation_data = datagen_test.flow(newXtarget, labels_val,batch_size=24),
                 callbacks = [checkpoint, reduce_lr])

In [ ]:
plt.figure(figsize=(4,5), dpi=150)
plt.grid('on')
plt.title('Loss Function')
plt.plot(hist.history['loss'], 'b', lw=2, alpha=0.7, label='Training')
plt.plot(hist.history['val_loss'], 'r', lw=2, alpha=0.7, label='Val')
plt.legend(loc="upper right")

plt.show()